# Homework 7 Final


In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import initializers, Model
from tensorflow.keras.layers import Dense, Layer

import numpy as np
import matplotlib.pyplot as plt

import random
from tqdm import tqdm
import time 

In [ ]:
# Defining Hyperparameters
SHUFFLE_SIZE = 1000
BATCH_SIZE = 64
PREFETCH_SIZE = 20

# Creating a Generation Function and Wrapper

In [ ]:
# Generate random noise 
def my_integration_task(seq_len, num_samples):
  num=0
  while num < num_samples:
    x=np.random.normal(0,1,seq_len)
    y=np.expand_dims(x,-1)
    num+=1
    sum=np.sum(x,axis=0)
    if (sum>1.0):
      target=1
      yield y,target
    else:
      target=0
      yield y,target


# create wrapper to feed to data_from_generator function of tensorflow
def integration_wrapper():
  for e in my_integration_task(25,80.000):
    yield e

# Create Dataset

In [ ]:

# create 2 datasets from generator

train_ds = tf.data.Dataset.from_generator(integration_wrapper, output_types=(tf.float32, tf.float32))

test_ds = tf.data.Dataset.from_generator(integration_wrapper, output_types=(tf.float32, tf.float32))

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2021-12-12 22:27:25.568541: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-12 22:27:25.568651: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Data Pipeline

In [ ]:
def preprocess(data):
    # Shuffle, batch and prefetch
    data = data.cache().shuffle(SHUFFLE_SIZE).batch(BATCH_SIZE).prefetch(PREFETCH_SIZE)
    return data
    

train_ds = preprocess(train_ds)
test_ds = preprocess(test_ds)

# LSTM Cell


In [ ]:
class LSTM_Cell(Layer):
  def __init__(self, units):
    super(LSTM_Cell, self).__init__()
    # define amount of units 
    self.units = units
    # forget_gate to drop values out of cell_state
    self.f_gate = tf.keras.layers.Dense(units, activation=tf.nn.sigmoid, bias_initializer=tf.keras.initializers.Ones)
    # input gate and candidates combined into updating cell-state
    self.input_gate = tf.keras.layers.Dense(units, activation=tf.nn.tanh, kernel_initializer='orthogonal')
    self.candidates_gate = tf.keras.layers.Dense(units, activation=tf.nn.sigmoid, kernel_initializer='orthogonal')
    # output filtering gate
    self.out_gate = tf.keras.layers.Dense(units, activation=tf.nn.sigmoid, kernel_initializer='orthogonal')
  

  
  def call(self, x, states):
    # split states 
    hidden_state, cell_state = states
    # concat hidden_state from t-1 with input at t
    concat_input = tf.concat((x, hidden_state), axis=-1)
    # apply forget_gate
    cell_state = cell_state*self.f_gate(concat_input)
    # compute update for cellstate
    update = self.input_gate(concat_input)*self.candidates_gate(concat_input)
    # update cell_state
    cell_state = cell_state + update
    # compute new hidden_state / output
    out = tf.nn.tanh(cell_state)*self.out_gate(concat_input)
    return out, (out, cell_state)

# LSTM Layer

In [ ]:

class LSTM(Layer):
  # initialize the class with cell
  def __init__(self, cell):
    super(LSTM, self).__init__()

    self.cell = cell
  

  # call the layer with mutliple time step (seq_len)  
  def call(self, x, states):
    seq_len = tf.shape(x)[1]
    outs = tf.TensorArray(dtype=tf.float32, size=seq_len, clear_after_read=True)
  	# for every timestep t
    for t in tf.range(seq_len):
      # compute otuput and hidden_state
      t_out, states = self.cell(x[:,t,:], states)
      outs = outs.write(t, t_out)
    
    # transpose and stack the output to return it in [batch_size, seq_len, output_size]
    out = outs.stack()
    out = tf.transpose(out, perm=[1,0,2])
    
    return out

  # function to reset/return to a zero_state, 
  def zero_state(self, batch_size):
    return (tf.zeros((batch_size, self.cell.units)), tf.zeros((batch_size, self.cell.units)))

# Model

In [ ]:
class RNN_Model(Model):
    def __init__(self):
        super(RNN_Model,self).__init__()
        # create 2 filtering input layers
        self.input_layer = Dense(64, activation="sigmoid")
        self.input_layer2 = Dense(32)
        # define a cell with amount of units and asssign the wrapper to it
        self.cell= LSTM_Cell(2)
        self.lstm_wrapper = LSTM(self.cell)

        # activation layer for binary output
        self.out = Dense(1, activation="sigmoid")



    def call(self, input):
        # get input shape and create initial zero_state for LSTM
        batch_size = tf.shape(input)[0]
        zero_state = self.lstm_wrapper.zero_state(batch_size=batch_size)
        # feed input through network layers
        x = self.input_layer(input) 
        x = self.input_layer2(x)
        x = self.lstm_wrapper(x, zero_state)
        x = self.out(x)
        return x

# Accucary Test

In [ ]:
# modified accuracy function to fit dimensions
@tf.function 
def train_step(model, input, target, loss_function, optimizer, training=True):
  target = tf.expand_dims(target, axis=-1)
  with tf.GradientTape(persistent=True) as tape:
    prediction = model(input)
    loss = loss_function(target, prediction[:,-1,:])
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  acc = tf.reduce_mean(tf.cast(tf.round(prediction[:,-1,:])==target, dtype=tf.float32))
  return loss, acc

# Training

In [ ]:
# define amount of epochs
iters = 100
#create model instance and define parameters
model = RNN_Model()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_function = tf.losses.BinaryCrossentropy()


# training loop
for iter in range(iters):
  loss_agg = []
  acc_agg = []
  for input, target in train_ds:
    loss, acc = train_step(model, input, target, loss_function, optimizer)
    loss_agg.append(loss)
    acc_agg.append(acc)
  print("Loss is %2.5f. Accuracy %2.5f" % (np.mean(loss_agg), np.mean(acc_agg)))

2021-12-12 22:27:27.405052: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-12 22:27:27.405217: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-12 22:27:28.595841: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-12 22:27:32.584476: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Loss is 0.68233. Accuracy 0.58594
Loss is 0.67536. Accuracy 0.60938
Loss is 0.69624. Accuracy 0.51562
Loss is 0.67357. Accuracy 0.60938
Loss is 0.69077. Accuracy 0.53906
Loss is 0.70291. Accuracy 0.49219
Loss is 0.70251. Accuracy 0.49219
Loss is 0.69511. Accuracy 0.51562
Loss is 0.68043. Accuracy 0.58594
Loss is 0.69317. Accuracy 0.51562
Loss is 0.68859. Accuracy 0.53906
Loss is 0.69786. Accuracy 0.46875
Loss is 0.68785. Accuracy 0.53906
Loss is 0.68732. Accuracy 0.53906
Loss is 0.68899. Accuracy 0.51562
Loss is 0.68805. Accuracy 0.58594
Loss is 0.68831. Accuracy 0.51562
Loss is 0.69040. Accuracy 0.51562
Loss is 0.68503. Accuracy 0.56250
Loss is 0.69401. Accuracy 0.46875
Loss is 0.68982. Accuracy 0.51562
Loss is 0.69030. Accuracy 0.51562
Loss is 0.69121. Accuracy 0.46875
Loss is 0.68802. Accuracy 0.54688
Loss is 0.68814. Accuracy 0.51562
Loss is 0.69046. Accuracy 0.55469
Loss is 0.69039. Accuracy 0.55469
Loss is 0.68501. Accuracy 0.67188
Loss is 0.68374. Accuracy 0.51562
Loss is 0.6863